<a href="https://colab.research.google.com/github/githubartema/root/blob/master/NIMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip3 install pytorch-lightning

     |████████████████████████████████| 256kB 2.7MB/s 
     |████████████████████████████████| 829kB 7.9MB/s 
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=f22a791fd5726b9b4304255da3f55c0a93c6bc3fa1ca9043ffff76c06081b86f
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
  Found existing installation: future 0.16.0
    Uninstalling future-0.16.0:
      Successfully uninstalled future-0.16.0


In [0]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from torchvision import transforms
import torchvision.models as models

import pytorch_lightning as pl
from pytorch_lightning import Trainer

import pandas as pd
from PIL import Image
import os

class NNetwork(pl.LightningModule):
    
    def __init__(self, base_model):
        super(NNetwork, self).__init__()
        self.features = base_model.features
        #for _ in self.features.parameters():
        #    _.requires_grad = False
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.1),
            nn.Linear(in_features=25088, out_features=10),
            nn.Softmax(dim=1))
    
    def forward(self, x):   
        x.requires_grad = True 
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x
    
    def training_step(self, batch, batch_idx):
        x, y = batch['image'].to(device), batch['annotations'].to(device).float()
        predicts = self(x)
        predicts = predicts.view(-1, 10, 1) 
        loss = self.emd_loss(predicts, y)
        loss = torch.autograd.Variable(loss, requires_grad = True)
        print(loss)
        tensorboard_logs = {'train_loss': loss}
        return {'loss': loss, 'log': tensorboard_logs}
    
    def validation_step(self, batch, batch_idx):
        x, y = batch['image'].to(device), batch['annotations'].to(device).float()
        predicts = self(x)
        predicts = predicts.view(-1, 10, 1)
        loss = self.emd_loss(predicts, y)
        return {'val_loss': loss}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        tensorboard_logs = {'val_loss': avg_loss}
        return {'val_loss': avg_loss, 'log': tensorboard_logs}
    
    def configure_optimizers(self):
        return torch.optim.Adam([
                    {'params': model.features.parameters(), 'lr': 0.001},
                    {'params': model.classifier.parameters(), 'lr': 0.001}])
    
    def prepare_data(self):
        train_transform = transforms.Compose([
            transforms.Scale(256),
            transforms.RandomCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor()])
    
        val_transform = transforms.Compose([
            transforms.Scale(256),
            transforms.RandomCrop(224),
            transforms.ToTensor()])
    
        self.trainset = GetDataset(csv_file='/content/sample_data/train_set.csv', root_dir='/content/sample_data/images', transform=train_transform)
        self.valset = GetDataset(csv_file='/content/sample_data/val_set.csv', root_dir='/content/sample_data/images', transform=val_transform)
       
    def train_dataloader(self):
        loader = DataLoader(self.trainset, batch_size=64, shuffle=True, num_workers=0)
        return loader
    
    def val_dataloader(self):
        loader = DataLoader(self.valset, batch_size=64, num_workers=0)
        return loader
         
    def single_emd_loss(self, p, q, r=2):
        emd_loss = 0.0
        length = p.shape[0]
        for _ in range(1, length + 1):
            emd_loss += torch.abs(sum(p[:_] - q[:_])) ** r
        return (emd_loss / length) ** (1. / r)
    
    def emd_loss(self, p, q, r=2):
        mini_batch_size = p.shape[0]
        loss_vector = []  
        for _ in range(mini_batch_size):
            loss_vector.append(self.single_emd_loss(p[_], q[_], r=r))
        return sum(loss_vector) / mini_batch_size
    
    
class GetDataset(torch.utils.data.Dataset):

    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, str(self.annotations.iloc[idx, 0]) + '.jpg')
        image = Image.open(img_name)
        annotations = self.annotations.iloc[idx, 1:].to_numpy()
        annotations = annotations.astype('float').reshape(-1, 1)
        sample = {'image': image, 'annotations': annotations}

        if self.transform:
            sample['image'] = self.transform(sample['image'])
        return sample


if __name__ == "__main__":
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    base_model = models.vgg16(pretrained=True)
    model = NNetwork(base_model)
    model = model.to(device)

    trainer = Trainer(progress_bar_refresh_rate=10, max_epochs=7)
    trainer.fit(model)  

        
        
    

GPU available: False, used: False
No environment variable for node rank defined. Set as 0.
/usr/local/lib/python3.6/dist-packages/torchvision/transforms/transforms.py:211: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")

   | Name         | Type       | Params
----------------------------------------
0  | features     | Sequential | 14 M  
1  | features.0   | Conv2d     | 1 K   
2  | features.1   | ReLU       | 0     
3  | features.2   | Conv2d     | 36 K  
4  | features.3   | ReLU       | 0     
5  | features.4   | MaxPool2d  | 0     
6  | features.5   | Conv2d     | 73 K  
7  | features.6   | ReLU       | 0     
8  | features.7   | Conv2d     | 147 K 
9  | features.8   | ReLU       | 0     
10 | features.9   | MaxPool2d  | 0     
11 | features.10  | Conv2d     | 295 K 
12 | features.11  | ReLU       | 0     
13 | features.12  | Conv2d     | 590 K 
14 | features.13  | ReLU       | 0 

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


tensor([0.1928], requires_grad=True)
tensor([0.1903], requires_grad=True)
tensor([0.1913], requires_grad=True)
tensor([0.1847], requires_grad=True)
tensor([0.1910], requires_grad=True)


/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning: Did not find hyperparameters at model hparams. Saving checkpoint without hyperparameters.
  warnings.warn(*args, **kwargs)


tensor([0.1868], requires_grad=True)
tensor([0.1936], requires_grad=True)
tensor([0.1878], requires_grad=True)
tensor([0.1894], requires_grad=True)
tensor([0.1875], requires_grad=True)


Detected KeyboardInterrupt, attempting graceful shutdown...


In [0]:
%reload_ext tensorboard
%tensorboard --logdir lightning_logs/version_2/

In case of using MobileNet or MNASNet we need to use the right dimensionality of fully connected layers in classifier. 